In [8]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.datasets import fetch_openml
import pickle
from scipy.special import expit
from sklearn.model_selection import train_test_split
from scipy.special import softmax
from joblib import Parallel, delayed
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

## Step 1
Load the MNIST dataset. Split this dataset into training (70%), validation (10%), and test dataset
(20%).



In [3]:

# Load MNIST data
X, y = fetch_openml('mnist_784', version=1, return_X_y=True, as_frame=False)
y = y.astype(int)

# Split the dataset
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=1)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=(2/3), random_state=1)

# Add bias term to the features
X_train = np.hstack([np.ones((X_train.shape[0], 1)), X_train])
X_val = np.hstack([np.ones((X_val.shape[0], 1)), X_val])
X_test = np.hstack([np.ones((X_test.shape[0], 1)), X_test])

# Number of classes
n_classes = len(np.unique(y))

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)


# Testing Function with Covariance
def testGaussianMCClassifier(X, y, A, cov_inv):
    scores = np.zeros((X.shape[0], n_classes))
    for i in range(n_classes):
        diff = X - A[:, i]
        scores[:, i] = -0.5 * np.sum(diff @ cov_inv[:, :, i] * diff, axis=1)
    y_pred = np.argmax(scores, axis=1)
    misclassifications = np.sum(y_pred != y)
    return misclassifications

c:\Users\maxfi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\datasets\_openml.py:1002: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


# Step 2
Consider the multi-class Gaussian classifier in page 23 of Logistic regression slide deck. Assume a
common covariance matrix Σ = I. Estimate the parameters μi; i = 0, .., n − 1 from the training
dataset. Develop a python function called gaussianMultiChannelClassifier that implements the
above classifier. This function accepts the training dataset and the labels, and returns the matrix A1,
whose columns are ai that corresponds to discriminants yi(x)

In [9]:
# Data normalization
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Dimensionality Reduction with PCA
pca = PCA(n_components=0.95)  # retain 95% variance
X_train_pca = pca.fit_transform(X_train_scaled)
X_val_pca = pca.transform(X_val_scaled)
X_test_pca = pca.transform(X_test_scaled)

# Gaussian Multi-Channel Classifier with Covariance
def gaussianMultiChannelClassifierPCA(Xtrain, ytrain, reg_lambda=0.8):
    A = np.zeros((Xtrain.shape[1], n_classes))
    cov_inv = np.zeros((Xtrain.shape[1], Xtrain.shape[1], n_classes))
    for i in range(n_classes):
        X_i = Xtrain[ytrain == i]
        mu_i = np.mean(X_i, axis=0)
        cov_i = np.cov(X_i, rowvar=False) + reg_lambda*np.identity(Xtrain.shape[1])
        cov_inv[:, :, i] = np.linalg.inv(cov_i)
        A[:, i] = np.dot(cov_inv[:, :, i], mu_i)
    return A, cov_inv

def find_best_lambda(X_train, y_train, X_val, y_val, lambda_values):
    best_lambda = None
    lowest_error = float('inf')

    for lambda_val in lambda_values:
        # Train the classifier with the current lambda value
        A, cov_inv = gaussianMultiChannelClassifier(X_train, y_train, reg_lambda=lambda_val)
        
        # Evaluate on the validation set
        misclassifications = testGaussianMCClassifier(X_val, y_val, A, cov_inv)
        percent_error = (misclassifications / X_val.shape[0]) * 100

        # Check if this is the best lambda so far
        if percent_error < lowest_error:
            best_lambda = lambda_val
            lowest_error = percent_error

        print(f"Lambda: {lambda_val}, Validation Error: {percent_error:.2f}%")

    return best_lambda

# 
# Fine tune stage 1
# lambda_values = [0.001, 0.01, 0.1, 1, 10, 100]

# Fine tune stage 2
#lambda_values = [0.5, 0.8, 0.9, 1, 1.1, 1.2, 1.5]

# Fine tine stage 3
# I created an array of lambda values evenly spaced between 0.8 and 1
#lambda_values = np.linspace(0.8, 1, 100)

# Find the best lambda
# best_lambda = find_best_lambda(X_train_scaled, y_train, X_val_scaled, y_val, lambda_values)
# best_lambda = find_best_lambda(X_train_scaled, y_train, X_val_scaled, y_val, lambda_values)
# print(f"Best Lambda: {best_lambda}")


# Step 3
implement a multiclass logistic regression algorithm to this dataset. Develop a python function called
logisticRegressionMultiClassClassifier that implements the above classifier. This function ac-
cepts the training dataset and the labels, and returns the matrix A2.

In [13]:
# 3. Multiclass Logistic Regression Classifier
def logisticRegressionMultiClassClassifier(Xtrain, ytrain, iterations=500, learning_rate=1e-4):
    A = np.zeros((Xtrain.shape[1], n_classes))
    for i in range(iterations):
        scores = np.dot(Xtrain, A)
        probs = softmax(scores, axis=1)
        y_one_hot = np.eye(n_classes)[ytrain]
        gradient = np.dot(Xtrain.T, (probs - y_one_hot)) / Xtrain.shape[0]
        A -= learning_rate * gradient
    return A

## Step 4
implement a regularized version of multi-class logistic regression. Develop a python function called
logisticRegressionMultiClassClassifierWithRegularization that implements the above classi-
fier. This function accept the training dataset, labels, and λ and return the matrix A3. You may test
this function independently with a specific λ value. 

In [14]:
# 4. Regularized Multiclass Logistic Regression
def logisticRegressionMultiClassClassifierWithRegularization(Xtrain, ytrain, lam=0.1, iterations=500, learning_rate=1e-4):
    A = np.zeros((Xtrain.shape[1], n_classes))
    for i in range(iterations):
        scores = np.dot(Xtrain, A)
        probs = softmax(scores, axis=1)
        y_one_hot = np.eye(n_classes)[ytrain]
        gradient = np.dot(Xtrain.T, (probs - y_one_hot)) / Xtrain.shape[0] + lam * A
        A -= learning_rate * gradient
    return A

## Step 6 
Write a function testLinearMCClassifier to test the performance of a multiclass classifier specified
by its weights A. This function would accept the test dataset, labels, and A and would return the
number of misclassified points.

In [15]:
# 6. Testing Functions
def testLinearMCClassifier(X, y, A):
    scores = np.dot(X, A)
    y_pred = np.argmax(scores, axis=1)
    misclassifications = np.sum(y_pred != y)
    return misclassifications



## Step 5
Write a function Optimize_MC_Hyperparameters to determine the optimal λ of
logisticRegressionMultiClassClassifierWithRegularization. This function accepts the train-
ing dataset, validation dataset, and the labels, and returns the matrix (A_1)that corresponds to the
optimal logistic regression classifier.

In [16]:
# 5. Optimize Hyperparameters

# Multithreaded Lambda evaluation
def train_and_evaluate_lambda(Xtrain, ytrain, Xval, yval, lam, iterations, learning_rate):
    A = logisticRegressionMultiClassClassifierWithRegularization(Xtrain, ytrain, lam, iterations, learning_rate)
    error = testLinearMCClassifier(Xval, yval, A)
    return lam, error, A

def Optimize_MC_Hyperparameters(Xtrain, ytrain, Xval, yval, iterations=500, learning_rate=1e-4):
    results = Parallel(n_jobs=-1)(delayed(train_and_evaluate_lambda)(
        Xtrain, ytrain, Xval, yval, lam, iterations, learning_rate) for lam in np.logspace(-4, 2, 20))
    
    best_result = min(results, key=lambda x: x[1])
    print(f'Best lambda: {best_result[0]}')
    return best_result[2]

# Step 7

Write a script that would compare the performance all the above algorithms on the test subset. This
script would call testLinearMCClassifier with each A1, A2 and report the misclassifications
in each case.

In [17]:
# 7. Comparison Script
A1, cov_inv_A1 = gaussianMultiChannelClassifierPCA(X_train_pca, y_train)
A2 = logisticRegressionMultiClassClassifier(X_train, y_train, iterations=1000)
A3 = Optimize_MC_Hyperparameters(X_train, y_train, X_val, y_val, iterations=1000)

# Test classifiers (missclassifications)
misclassifications_A1 = testGaussianMCClassifier(X_test_pca, y_test, A1, cov_inv_A1)
misclassifications_A2 = testLinearMCClassifier(X_test, y_test, A2)
misclassifications_A3 = testLinearMCClassifier(X_test, y_test, A3)

# Calculate percentage errors
percent_error_A1 = (misclassifications_A1 / X_test_pca.shape[0]) * 100
percent_error_A2 = (misclassifications_A2 / X_test.shape[0]) * 100
percent_error_A3 = (misclassifications_A3 / X_test.shape[0]) * 100

# Print out the percentage errors
print(f"Gaussian Multi-Channel Classifier Percentage Error: {percent_error_A1:.2f}%")
print(f"Multiclass Logistic Regression Percentage Error: {percent_error_A2:.2f}%")
print(f"Optimized Multiclass Logistic Regression Percentage Error: {percent_error_A3:.2f}%")

# Example Output
# Best lambda: 0.007847599703514606
# Gaussian Multi-Channel Classifier Percentage Error: 8.44%
# Multiclass Logistic Regression Percentage Error: 8.39%
# Optimized Multiclass Logistic Regression Percentage Error: 8.26%

Best lambda: 0.007847599703514606
Gaussian Multi-Channel Classifier Percentage Error: 8.44%
Multiclass Logistic Regression Percentage Error: 8.39%
Optimized Multiclass Logistic Regression Percentage Error: 8.26%
